In [1]:
#RMS
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
df=pd.read_csv(r"C:\Users\2000139\ShurovoData_2018H2\DISPL_X_MAX_T_1.txt")
df.head()

,Number of points: 44266
0,Sampling rate(Hz): 0.00333333333333333
1,Time range: From: 03/06/2015 10:19:59; Until: ...
2,Units: mm
3,Description: DISPL_X_MAX_T
4,01.07.2018 00:01:05\t8.44


In [3]:
df.columns=['DISPL_X_MAX_T_1']
df = pd.DataFrame(df['DISPL_X_MAX_T_1'].str.split('\t',2).tolist())

In [4]:
df[df.isnull().any(axis=1)].head(50)

,0,1
0,Sampling rate(Hz): 0.00333333333333333,None
1,Time range: From: 03/06/2015 10:19:59; Until: ...,None
2,Units: mm,None
3,Description: DISPL_X_MAX_T,None


In [5]:
df.dropna(inplace=True)

In [6]:
df.head()

,0,1
4,01.07.2018 00:01:05,8.44
5,01.07.2018 00:06:05,8.429999
6,01.07.2018 00:11:05,8.37
7,01.07.2018 00:16:05,8.429999
8,01.07.2018 00:21:05,8.349999


In [7]:
    df[0]=pd.to_datetime(df[0])
    df.index=df[0]
    df.index=df.index.date
    df.drop(columns=[0],inplace=True)
    df.columns=['DISPL_X_MAX_T_1']
    df.isnull().sum()
    df=df.astype({'DISPL_X_MAX_T_1':float,})
    df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44266 entries, 2018-01-07 to 2018-12-31
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DISPL_X_MAX_T_1  44266 non-null  float64
dtypes: float64(1)
memory usage: 691.7+ KB


In [8]:
df.head()

,DISPL_X_MAX_T_1
2018-01-07,8.440000
2018-01-07,8.429999
2018-01-07,8.370000
2018-01-07,8.429999
2018-01-07,8.349999


In [9]:
df['square_values'] = df.DISPL_X_MAX_T_1*df.DISPL_X_MAX_T_1

In [10]:
df.head()

,DISPL_X_MAX_T_1,square_values
2018-01-07,8.440000,71.233600
2018-01-07,8.429999,71.064883
2018-01-07,8.370000,70.056900
2018-01-07,8.429999,71.064883
2018-01-07,8.349999,69.722483


In [11]:
final_df=df.groupby(df.index).agg({'square_values':['mean']})

In [12]:
final_df

,square_values
,mean
2018-01-07,68.893742
2018-01-09,67.959840
2018-01-10,67.960023
2018-01-11,60.840000
2018-01-12,66.446244
...,...
2018-12-27,66.703177
2018-12-28,63.602137
2018-12-29,61.312303


In [13]:
final_df
header = np.array(['RMS'])
final_df.columns=header

In [14]:
final_df['RMS'] = final_df['RMS'].apply(lambda x : x**0.5)
final_df

,RMS
2018-01-07,8.300225
2018-01-09,8.243776
2018-01-10,8.243787
2018-01-11,7.800000
2018-01-12,8.151457
...,...
2018-12-27,8.167201
2018-12-28,7.975095
2018-12-29,7.830217
2018-12-30,7.824342


In [15]:
outlier_df=final_df[final_df['RMS']<=0.1]
outlier_df.columns=['Non Running']

In [16]:
outlier_df.index

Index([2018-02-08, 2018-06-08, 2018-09-13, 2018-09-14], dtype='object')

In [17]:
outlier_df.index=pd.to_datetime(outlier_df.index)

In [18]:
final_df_1=final_df[:]
final_df_1

,RMS
2018-01-07,8.300225
2018-01-09,8.243776
2018-01-10,8.243787
2018-01-11,7.800000
2018-01-12,8.151457
...,...
2018-12-27,8.167201
2018-12-28,7.975095
2018-12-29,7.830217
2018-12-30,7.824342


In [19]:
final_df_1.index=final_df_1.index.astype('str')
final_df_1.drop(final_df_1[final_df_1['RMS'] <=0.1].index, inplace= True)
final_df_1.index=pd.to_datetime(final_df_1.index)

In [20]:
final_df_1

,RMS
2018-01-07,8.300225
2018-01-09,8.243776
2018-01-10,8.243787
2018-01-11,7.800000
2018-01-12,8.151457
...,...
2018-12-27,8.167201
2018-12-28,7.975095
2018-12-29,7.830217
2018-12-30,7.824342


In [21]:

fit1 = SimpleExpSmoothing(final_df_1['RMS']).fit(smoothing_level=0.2,optimized = False)
final_df_1['Smooth RMS']=list(fit1.fittedvalues)

C:\Users\2000139\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [22]:
final_df_1

,RMS,Smooth RMS
2018-01-07,8.300225,8.300225
2018-01-09,8.243776,8.300225
2018-01-10,8.243787,8.288935
2018-01-11,7.800000,8.279906
2018-01-12,8.151457,8.183925
...,...,...
2018-12-27,8.167201,8.144111
2018-12-28,7.975095,8.148729
2018-12-29,7.830217,8.114002
2018-12-30,7.824342,8.057245


In [23]:
pd.date_range(start="2018-01-07",end="2018-12-31", freq='h').to_pydatetime().tolist()

[datetime.datetime(2018, 1, 7, 0, 0),
 datetime.datetime(2018, 1, 7, 1, 0),
 datetime.datetime(2018, 1, 7, 2, 0),
 datetime.datetime(2018, 1, 7, 3, 0),
 datetime.datetime(2018, 1, 7, 4, 0),
 datetime.datetime(2018, 1, 7, 5, 0),
 datetime.datetime(2018, 1, 7, 6, 0),
 datetime.datetime(2018, 1, 7, 7, 0),
 datetime.datetime(2018, 1, 7, 8, 0),
 datetime.datetime(2018, 1, 7, 9, 0),
 datetime.datetime(2018, 1, 7, 10, 0),
 datetime.datetime(2018, 1, 7, 11, 0),
 datetime.datetime(2018, 1, 7, 12, 0),
 datetime.datetime(2018, 1, 7, 13, 0),
 datetime.datetime(2018, 1, 7, 14, 0),
 datetime.datetime(2018, 1, 7, 15, 0),
 datetime.datetime(2018, 1, 7, 16, 0),
 datetime.datetime(2018, 1, 7, 17, 0),
 datetime.datetime(2018, 1, 7, 18, 0),
 datetime.datetime(2018, 1, 7, 19, 0),
 datetime.datetime(2018, 1, 7, 20, 0),
 datetime.datetime(2018, 1, 7, 21, 0),
 datetime.datetime(2018, 1, 7, 22, 0),
 datetime.datetime(2018, 1, 7, 23, 0),
 datetime.datetime(2018, 1, 8, 0, 0),
 datetime.datetime(2018, 1, 8, 1, 0)

In [24]:
final_df_1.loc[datetime(2018, 1, 7)]

RMS           8.300225
Smooth RMS    8.300225
Name: 2018-01-07 00:00:00, dtype: float64

In [25]:
rms_df=pd.DataFrame(columns=['Smooth_RMS'],index=pd.date_range(start="2018-01-07",end="2018-12-31", freq='d').to_pydatetime().tolist())

In [26]:
# rms_df['Date']=pd.date_range(start="2018-01-07",end="2018-12-31", freq='d').to_pydatetime().tolist()

In [27]:
rms_df

,Smooth_RMS
2018-01-07,NaN
2018-01-08,NaN
2018-01-09,NaN
2018-01-10,NaN
2018-01-11,NaN
...,...
2018-12-27,NaN
2018-12-28,NaN
2018-12-29,NaN
2018-12-30,NaN


In [28]:
#pd.merge(left=final_df_1,right=rms_df,left_on='Smooth RMS', right_on='Smooth_RMS', left_index=False, right_index=True)
rms_df=pd.concat([final_df_1, rms_df,outlier_df], axis=1, join='outer').drop(columns=['Smooth_RMS'])

In [29]:
# pd.concat([final_df_1, rms_df,outlier_df], axis=1, join='outer')

In [30]:
rms_df

,RMS,Smooth RMS,Non Running
2018-01-07,8.300225,8.300225,NaN
2018-01-08,NaN,NaN,NaN
2018-01-09,8.243776,8.300225,NaN
2018-01-10,8.243787,8.288935,NaN
2018-01-11,7.800000,8.279906,NaN
...,...,...,...
2018-12-27,8.167201,8.144111,NaN
2018-12-28,7.975095,8.148729,NaN
2018-12-29,7.830217,8.114002,NaN
2018-12-30,7.824342,8.057245,NaN


In [31]:
rms_df['Smooth RMS']=rms_df['Smooth RMS'].interpolate('linear')

In [32]:
rms_df

,RMS,Smooth RMS,Non Running
2018-01-07,8.300225,8.300225,NaN
2018-01-08,NaN,8.300225,NaN
2018-01-09,8.243776,8.300225,NaN
2018-01-10,8.243787,8.288935,NaN
2018-01-11,7.800000,8.279906,NaN
...,...,...,...
2018-12-27,8.167201,8.144111,NaN
2018-12-28,7.975095,8.148729,NaN
2018-12-29,7.830217,8.114002,NaN
2018-12-30,7.824342,8.057245,NaN


In [1]:
fig=go.Figure()
#fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1['Smooth RMS'],mode='markers',name = 'Smooth_RMS'))
fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['Smooth RMS'].interpolate('polynomial',order=1),mode='markers',name = 'Smooth_RMS'))
#fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1['Smooth RMS'],mode='markers',name = 'Smooth_RMS',line=dict(color="#ffae42")))
fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['Non Running'],mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
fig.add_trace(go.Scatter(x=rms_df.index,y=rms_df['RMS'],mode='markers',name = 'RMS',line=dict(color="#5dd55d")))
fig.update_layout(hovermode="x")
fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'RMS DATA','xanchor':'center','x':0.5})
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Amplitude')

NameError: name 'go' is not defined

In [34]:
rms_df.to_csv('RMS_DISPL_X_MAX_T_1.csv')

In [35]:
fig=go.Figure()
#fig.add_trace(go.Scatter(x=RMS_copy.index,y=RMS_copy.RMS,name='RMS'))
px.scatter(final_df.RMS,labels={'x':'Date','y':'Value'})


In [36]:
fig=go.Figure()
#fig.add_trace(go.Scatter(x=RMS_copy.index,y=RMS_copy.RMS,name='RMS'))
px.line(final_df.RMS,labels={'x':'Date','y':'Value'})


In [37]:
final_df.to_csv(r'stastical_output\RMS\\DISPL_X_MAX_T_1.csv',header=True)
#fig.write_html("stastical_output\RMS\\"+'DISPL_X_MAX_T_1'+".html")